# Generate some data
The script is copied is from
http://amp.readthedocs.io/en/latest/examplescripts.html,
We used a larger cell due to the limited PBC implementation in our code.

In [ ]:
import os
from ase import Atoms, Atom, units
import ase.io
from ase.calculators.emt import EMT
from ase.build.surface import fcc110
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md import VelocityVerlet
from ase.constraints import FixAtoms
import nglview as nv

def generate_data(count, filename='training.traj'):
    """Generates test or training data with a simple MD simulation."""
    if os.path.exists(filename):
        os.remove(filename)
    traj = ase.io.Trajectory(filename, 'w')
    atoms = fcc110('Pt', (4, 5, 2), vacuum=7.)
    atoms.extend(Atoms([Atom('Cu', atoms[7].position + (0., 0., 2.5)),
                        Atom('Cu', atoms[7].position + (0., 0., 5.)),
                        Atom('Cu', atoms[17].position + (0., 0., 2.5)),
                        Atom('Cu', atoms[17].position + (0., 0., 5.))]))
    atoms.set_constraint(FixAtoms(indices=[0, 2]))
    atoms.set_calculator(EMT())
    atoms.get_potential_energy()
    traj.write(atoms)
    MaxwellBoltzmannDistribution(atoms, 300. * units.kB)
    dyn = VelocityVerlet(atoms, dt=1. * units.fs)
    for step in range(count - 1):
        dyn.run(50)
        traj.write(atoms)

generate_data(20)

# Training

In [ ]:
from bpnn import BPNN

In [ ]:
from ase.io import read
import nglview as nv
traj = read('training.traj',index=':')
nv.show_asetraj(traj)

In [ ]:
calc = BPNN(symbols=['Pt','Cu'])
costs = calc.train(dataset=[traj],max_steps=1000,learning_rate=0.03)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(costs[100:])